In [24]:
import json
import math

tension_bonus = {
    'perfect': 1.04,
    'great': 1.02,
    'normal': 1,
    'bad': 0.98,
    'terrible': 0.96
}

course_bonus = {
    'acc':{
        'B': 0.9,
        'A': 1,
        'S': 1.05
    }
}

distance_bonus = {
    'speed':{
        'B': 0.9,
        'A': 1,
        'S': 1.05
    },
    'acc':{
        'B': 1,
        'A': 1,
        'S': 1
    }
}

tactic_bonus = {
    'smart':{
        'B': 0.85,
        'A': 1,
        'S': 1.1
    }
}


tactic_param ={
    'nige':{
        'speed':{
            'first': 1, 
            'middle': 0.98, 
            'last': 0.962,
            'final_part': 0.962
        },
        'acc':{
            'first': 1, 
            'middle': 1, 
            'last': 0.996,
            'final_part': 0.996
        }
    },
    'senkou':{
        'speed':{
            'first': 0.978, 
            'middle': 0.991, 
            'last': 0.975,
            'final_part': 0.975
        },
        'acc':{
            'first': 0.985, 
            'middle': 1, 
            'last': 0.996,
            'final_part': 0.996
        }
    },
    'sashi':{
        'speed':{
            'first': 0.938, 
            'middle': 0.998, 
            'last': 0.994,
            'final_part': 0.994
        },
        'acc':{
            'first': 0.975, 
            'middle': 1, 
            'last': 1,
            'final_part': 1
        }
    }, 
    'oikomu':{
        'speed':{
            'first': 0.931, 
            'middle': 1, 
            'last': 1,
            'final_part': 1
        },
        'acc':{
            'first': 0.945, 
            'middle': 1, 
            'last': 0.997,
            'final_part': 0.997
        }
    }

}

class Uma:
    def __init__(self, attr, aptitude):
        self.__attr = attr
        self.__aptitude = aptitude
        
    def CalAttr(self, tension, tactic_type, distance_type, course_type):
        
        self.status = {
            'speed': self.__attr['speed'] * tension_bonus[tension],
            'power': self.__attr['power'] * tension_bonus[tension],
            'smart': self.__attr['smart'] * tension_bonus[tension] * tactic_bonus['smart'][self.__aptitude['tactic'][tactic_type]],
            'tactic': {
                'type': tactic_type,
                'speed': tactic_param[tactic_type]['speed'],
                'acc': tactic_param[tactic_type]['acc']
            },
            'course_bonus':{
                'acc':course_bonus['acc'][self.__aptitude['course'][course_type]]
            },
            'distance_bonus':{
                'speed':distance_bonus['speed'][self.__aptitude['distance'][distance_type]],
                'acc':distance_bonus['acc'][self.__aptitude['distance'][distance_type]]
            }

        }
    
    def CalBaseSpeed(self, distance):
        
        

        ### 不受賽道影響的參數
        
        # 智力影響下限=−0.65+(智力/5500)×log(智力×0.1) [%]
        # 智力影響上限=(智力/5500)×log(智力×0.1) [%]
        
        smart = self.__attr['smart']
        
        speed_variation_by_smart_top = (smart / 5500) * math.log(smart * 0.1, 10)
        speed_variation_by_smart_bottom  = speed_variation_by_smart_top - 0.65
        speed_variation_by_smart_avg = speed_variation_by_smart_top - 0.325

        speed_variation_by_smart_avg = speed_variation_by_smart_avg * 0.01    
        
        
        ### 受賽道影響的參數
        # 基準速度 = 20.0 - (賽場距離[m] - 2000) / 1000 [m/s]
        
        track_base_speed = 22 - distance / 1000

        
        
        ### 計算基礎目標速度

        # 基礎目標速度(序盤)=比賽基準速度×(序盤跑法影響+智力影響)+(技能影響)
        # 基礎目標速度(中盤)=比賽基準速度×(中盤跑法影響+智力影響)+(技能影響)
        # 基礎目標速度(終盤)=比賽基準速度×(終盤跑法影響+智力影響)+(技能影響) + 終盤速度參數
        
        # 基礎目標速度(最終衝刺)=(基礎目標速度(終盤)+0.01×比賽基準速度)×1.05+(技能影響) + 終盤速度參數
        
        # 終盤速度參數 = √速度屬性×500×距性係數×0.002

        tactic = self.status['tactic']
        distance_bonus = self.status['distance_bonus']['speed']
        
        base_speed = {
            'first': track_base_speed * (tactic['speed']['first'] + speed_variation_by_smart_avg), 
            'middle': track_base_speed * (tactic['speed']['middle'] + speed_variation_by_smart_avg), 
            'last': track_base_speed * (tactic['speed']['last'] + speed_variation_by_smart_avg),
        }
        
        speed_constant = math.sqrt(self.status['speed'] * 500) * distance_bonus * 0.002
        base_speed['last'] += speed_constant
        base_speed['final_part'] = (base_speed['last'] + 0.01 * track_base_speed) * 1.05
        base_speed['final_part'] += speed_constant
        
        self.base_speed = base_speed
        
    def CalBaseAcc(self):
        
        # 加速度（加速）加速度 = 0.0006×√力量×500×跑法影響×草泥影響×距離適性係數+(技能影響)
        # 上坡時為0.0004
        # 起跑時提供24的加速度補正
        
        # 加速度（減速）
        # 依照狀況分為4個項目：
        # 序盤：-1.2
        # 中盤：-0.8
        # 終盤：-1.0
        # 耗盡體力時：-1.2
        
        power = self.status['power']
        tactic_acc = self.status['tactic']['acc']
        course_bonus_acc = self.status['course_bonus']['acc']
        
        acc_temp = math.sqrt(power * 500) * course_bonus_acc
        acc_normal = acc_temp * 0.0006
        acc_saka = acc_temp * 0.0004
        
        self.acc = {
            'first': {
                'normal': acc_normal * tactic_acc['first'],
                'saka': acc_saka * tactic_acc['first'],
                'minus': -1.2
            }, 
            'middle': {
                'normal': acc_normal * tactic_acc['middle'],
                'saka': acc_saka * tactic_acc['middle'],
                'minus': -0.8
            }, 
            'last': {
                'normal': acc_normal * tactic_acc['last'],
                'saka': acc_saka * tactic_acc['last'],
                'minus': -1
            },
            'final_part': {
                'normal': acc_normal * tactic_acc['final_part'],
                'saka': acc_saka * tactic_acc['final_part'],
                'minus': -1,
                'exhausted': -1.2
            }
        }
    
        self.acc['start_dash'] = 24 + self.acc['first']['normal']
    
class Track:
    def __init__(self, distance, name):
        self.__distance = distance
        self.__name = name
    def LoadTrackData(self):
        with open(self.__name + '.json') as track_file:
            self.track_data = json.load(track_file)
        
        return self.track_data
    
class Race:
    def __init__(self, uma, track_data):
        self.uma = uma
        self.track_data = track_data
        


        
        
uma = Uma(
    {        
        'speed': 1200,
        'power': 900,
        'smart': 400
    },
    {
        'course': {
            'shiba': 'A',
            'dirt': 'E',
        },
        'distance': {
            'sprint': 'D',
            'mile': 'D',
            'middle': 'A',
            'staying': 'A'
        },
        'tactic': {
            'nige': 'A',
            'senkou': 'D',
            'sashi': 'D',
            'oikomu': 'D'
        }
    }
)

track = Track(3200, 'Kyoto_outside_double')
# race = Race(uma.CalAttr('perfect', 'nige', 'staying', 'shiba'), track.LoadTrackData())

uma.CalAttr('perfect', 'nige', 'staying', 'shiba')
uma.CalBaseSpeed(3200)
print(uma.status)
print(uma.base_speed)
# print(uma.GetAttr())

{'speed': 1248.0, 'power': 936.0, 'smart': 416.0, 'tactic': {'type': 'nige', 'speed': {'first': 1, 'middle': 0.98, 'last': 0.962, 'final_part': 0.962}, 'acc': {'first': 1, 'middle': 1, 'last': 0.996, 'final_part': 0.996}}, 'course_bonus': {'acc': 1}, 'distance_bonus': {'speed': 1, 'acc': 1}}
{'first': 18.760804529335974, 'middle': 18.384804529335977, 'last': 19.626277941986494, 'final_part': 22.38486525173634}


In [14]:
string = """加速度（減速）
依照狀況分為4個項目：

序盤：-1.2
中盤：-0.8
終盤：-1.0
耗盡體力時：-1.2
"""

print(string.replace('\n', ''))

加速度（加速）加速度= 0.0006∗×√力量×500×跑法影響×草泥影響×距離適性係數+(技能影響)∗上坡時此項為0.0004起跑時提供24的加速度補正跑法影響／草泥影響／距離適性。
